In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm

# Suppress "SettingWithCopyWarning"
pd.options.mode.chained_assignment = None 

In [ ]:
clean_spacy_mapaffil = pd.read_parquet("data/clean_spacy_mapaffil.parquet", engine="fastparquet") 

In [ ]:
num_affiliations = 15000

In [ ]:
df = clean_spacy_mapaffil.head(num_affiliations)
city_counts = df['city'].value_counts()
single_instance_cities = city_counts[city_counts == 1].index.tolist()
num_affiliations -= len(single_instance_cities)
filtered_df = df[~df['city'].isin(single_instance_cities)]

In [ ]:
filtered_df['city'] = filtered_df['city'].astype('category')
filtered_df['label'] = filtered_df['city'].cat.codes

In [ ]:
calculated_test_size = (filtered_df['city'].nunique()) / num_affiliations
X_train_texts, X_test_texts, y_train, y_test = train_test_split(filtered_df["affiliation"], filtered_df["city"], test_size=calculated_test_size if calculated_test_size > 0.1 else 0.1, stratify=filtered_df['label'], random_state=42)

In [ ]:
# TF-IDF Vectorizer 
tfidf_vectorizer = TfidfVectorizer(stop_words="english", decode_error="ignore") 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_texts)
X_test_tfidf = tfidf_vectorizer.transform(X_test_texts)

In [ ]:
clf = svm.LinearSVC(dual=True)

def train_and_evaluate(X_train, X_test, y_train, y_test, vectorizer_name):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
accuracy_tfidf = train_and_evaluate(X_train_tfidf, X_test_tfidf, y_train, y_test, 'TF-IDF')

In [ ]:
results = pd.DataFrame({
    'Vectorizer': ['TF-IDF'],
    'Accuracy': [accuracy_tfidf]
})

display(results)